# Create Python Recommender System Using Cosine Similarity
- Recommend Next Best Action for Call Center Customer Interactions

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from datetime import date
import random

df = pd.read_csv('Data_orig/call_center_v2.csv',sep= ',')

df["RESPONSE_DATE"] = str(np.random.choice(pd.date_range('2018-01-01', '2021-11-01')))[0:10]

df=df.sample(frac=1).reset_index(drop=True)

df['index']= df.index

print(df.head())
 
def action_if(x): 
    if (x < 10 ):
        return ' '
    elif (x < 604):
        return 'Contact Account Managment'
    elif (x < 1199):
        return 'Contact Product Managment'
    elif (x < 1492):
        return 'Offer Discount'
    elif (x < 1794):
        return 'Dismiss'
    elif (x < 2389):
        return 'Verify Product Defect History'
    elif (x < 2984):
        return 'Verify Purchase History'
    elif (x < 3579):
        return 'Defer Unitl Next Week'
    elif (x < 4175):
        return 'Call Again in 3 Days'
    elif (x < 4771):
        return 'Verify Account Info'
    else:
        return 'Send to Collections'
    
action_group = df.copy()   
df['ACTION'] = action_group['index'].apply(action_if)

 
#df2 = df.fillna(0)
#df = pd.concat([df1,df2], sort=False)


df2 =df.fillna(0)
df3 = df2.drop(['index', 'RESPONSE_DATE', 'ESCALATED', 'ACTION'], axis=1)

 
df_scoring = df3[0:10]
df_history = df2.drop(['index', 'ESCALATED'], axis=1)[10:]

print(df2.dtypes)

dfgene =df2[['DAILY_CALLS', 'MONTHLY_CALLS', 'CALL_TARGET', 'MONTHS_CUSTOMER','NEGATIVE_RATING','DAYS_PROD_OUT_OF_SERVICE','PRODUCT_AGE', 'CALL_INQUIRIES', 'NUM_CALLERS', 'CALL_TIME']].to_numpy(dtype= 'float32')
dfgene3 =df2[['DAILY_CALLS', 'MONTHLY_CALLS', 'CALL_TARGET', 'MONTHS_CUSTOMER','NEGATIVE_RATING','DAYS_PROD_OUT_OF_SERVICE','PRODUCT_AGE', 'CALL_INQUIRIES', 'NUM_CALLERS', 'CALL_TIME']]

 
# #write out remediation history and new failures
####df_history.to_csv('/sasinside/userdata/gegrab/resources/hmeq/callcenter_remediation_history.csv', index=False)
####df_scoring.to_csv('Data_orig/callcenter_scoring.csv', index=False)


 
list(df_scoring)

   CALL_CENTER_ID  ESCALATED  DAILY_CALLS  MONTHLY_CALLS  CALL_TARGET  \
0          576937          0          315          108.0        580.0   
1          398647          0          134          679.0        849.0   
2          877065          0          176          648.0        824.0   
3          745512          0           93          440.0        705.0   
4          170162          0          242          906.0       1166.0   

        REASON  MONTHS_CUSTOMER  NEGATIVE_RATING  DAYS_PROD_OUT_OF_SERVICE  \
0  ProdFailure              0.0              0.0                       0.0   
1  ProdFailure              6.0              1.0                       3.0   
2  ProdFailure              1.0              0.0                       0.0   
3   ProdGlitch             12.0              3.0                       1.0   
4  ProdFailure              2.0              0.0                       0.0   

   PRODUCT_AGE  CALL_INQUIRIES  NUM_CALLERS  CALL_TIME PRODUCT_TYPE  \
0    94.248036       

['CALL_CENTER_ID',
 'DAILY_CALLS',
 'MONTHLY_CALLS',
 'CALL_TARGET',
 'REASON',
 'MONTHS_CUSTOMER',
 'NEGATIVE_RATING',
 'DAYS_PROD_OUT_OF_SERVICE',
 'PRODUCT_AGE',
 'CALL_INQUIRIES',
 'NUM_CALLERS',
 'CALL_TIME',
 'PRODUCT_TYPE',
 'FEEDBACK',
 'Sentiment']

In [88]:
list(df_scoring)

['CALL_CENTER_ID',
 'DAILY_CALLS',
 'MONTHLY_CALLS',
 'CALL_TARGET',
 'REASON',
 'MONTHS_CUSTOMER',
 'NEGATIVE_RATING',
 'DAYS_PROD_OUT_OF_SERVICE',
 'PRODUCT_AGE',
 'CALL_INQUIRIES',
 'NUM_CALLERS',
 'CALL_TIME',
 'PRODUCT_TYPE',
 'FEEDBACK',
 'Sentiment']

In [89]:
# def combined_features(row):
#    return str(row['DAILY_PRODUCTION'])+" "+str(row['MONTHLY_PRODUCTION'])+" "+str(row['PROD_TARGET'])+" "+str(row['REASON'])+" "+str(row['MONTHS_IN_OPERATION'])+" "+str(row['NEGATIVE_RATING'])+" "+str(row['DAYS_OUT_OF_SERVICE'])+" "+str(row['MACHINE_AGE'])+" "+str(row['SENSOR_INQUIRIES'])+" "+str(row['NUM_SENSORS'])+" "+str(row['SENSOR_RATIO'])+" "+str(row['INDUSTRY'])
# df["combined_features"] = df.apply(combined_features, axis =1)

# cv = CountVectorizer()
# count_matrix = cv.fit_transform(df2["FEEDBACK"])
# print("Count Matrix:", count_matrix.toarray())
#print("Count Matrix:", count_matrix)

# callcenter_index =0
# cosine_sim = cosine_similarity(count_matrix)
# similar_callcenter = list(enumerate(cosine_sim[callcenter_index])) 

# sorted_similar_callcenter = sorted(callcenter, key=lambda x:x[1], reverse=True)
# print(sorted_similar_callcenter[0:11])

In [90]:
callcenter_index =0
cosine_sim = cosine_similarity(dfgene)
similar_callcenter = list(enumerate(cosine_sim[callcenter_index])) 
sorted_similar_callcenter = sorted(similar_callcenter, key=lambda x:x[1], reverse=True)
print(sorted_similar_callcenter[0:11])
     

[(0, 1.0), (4367, 0.9999692), (5897, 0.99995434), (4429, 0.9999361), (2886, 0.9999202), (3453, 0.99989754), (5616, 0.99988276), (3110, 0.99987376), (1553, 0.99986815), (2259, 0.9998571), (953, 0.99985015)]


In [91]:
# Print remediation and date of machines that had the highest cosine simularity

def get_callcenter_from_index2(index):
    #return df[['MACHINE_ID' ,'FAILURE']]
    return (df2[df2.index == index]["CALL_CENTER_ID"].values[0], df2[df2.index == index]["ACTION"].values[0],
            df2[df2.index == index]["RESPONSE_DATE"].values[0])
            #sorted_similar_machines[i][1], sorted_similar_machines[i][0])
            
 

print('Call Centers IDs with Predicted Escalation:\n',df2[df2.index == 0]["CALL_CENTER_ID"].values[0]) 
print('\n')

i=0
print('"Call Center IDs with Similar Model Features and the Ensuing Response Action"')
print('Call Center IDs + Remediation + Remediation Date:\n')
for callcenter in sorted_similar_callcenter:
    i=i+1
    
    if i > 1:
        print(get_callcenter_from_index2(callcenter[0]))  
    if i  > 10:
        break
    
        




Call Centers IDs with Predicted Escalation:
 70541


"Call Center IDs with Similar Model Features and the Ensuing Response Action"
Call Center IDs + Remediation + Remediation Date:

(201898, 'Verify Account Info', '2021-06-18')
(76229, 'Send to Collections', '2021-06-18')
(19009, 'Verify Account Info', '2021-06-18')
(680364, 'Verify Purchase History', '2021-06-18')
(93266, 'Defer Unitl Next Week', '2021-06-18')
(502520, 'Send to Collections', '2021-06-18')
(215088, 'Defer Unitl Next Week', '2021-06-18')
(60861, 'Dismiss', '2021-06-18')
(807350, 'Verify Product Defect History', '2021-06-18')
(361083, 'Contact Product Managment', '2021-06-18')


In [92]:
# i=0
# for machine in sorted_similar_machines:
#     print(get_machine_from_index2(machine[0]))
#     i=i+1
#     if i  > 10:
#         break

# machine[0:2]

In [4]:
def execute (CALL_CENTER_ID,
            DAILY_CALLS,
            MONTHLY_CALLS,
            CALL_TARGET,
            REASON,
            MONTHS_CUSTOMER,
            NEGATIVE_RATING,
            DAYS_PROD_OUT_OF_SERVICE,
            PRODUCT_AGE,
            CALL_INQUIRIES,
            NUM_CALLERS,
            CALL_TIME,
            PRODUCT_TYPE,
            FEEDBACK,
            Sentiment) :
    "Output: COSINE_1, CALL_CENTER_ID_1, APP_DATE_1, ACTION_1"
             
    import pandas as pd
    import pickle
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID, DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING,
                                DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE, FEEDBACK, Sentiment]],         
                                
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS","MONTHLY_CALLS","CALL_TARGET","REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING",
                                         "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS","CALL_TIME", "PRODUCT_TYPE", "FEEDBACK", "Sentiment"]
                             )
    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray2 =inputArray[["DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "MONTHS_CUSTOMER","NEGATIVE_RATING",
                             "DAYS_PROD_OUT_OF_SERVICE","PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME"]]
    
    df = pd.read_csv('/sasinside/userdata/gegrab/resources/hmeq/callcenter_remediation_history.csv',sep= ',')
    df['index']= df.index+1
    
    df2 =df[["DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "MONTHS_CUSTOMER","NEGATIVE_RATING",
             "DAYS_PROD_OUT_OF_SERVICE","PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME"]]
    
    combined = pd.concat([inputArray2,df2], sort=False)
    callcenter_index =0
    cosine_sim = cosine_similarity(combined) 
    similar_callcenter = list(enumerate(cosine_sim[callcenter_index])) 
    sorted_similar_callcenter = pd.DataFrame(sorted(similar_callcenter, key=lambda x:x[1], reverse=True))
    sorted_similar_callcenter.columns  = ['index', 'cosine_sim']
    
    merged = pd.merge(sorted_similar_callcenter, df,on=['index'])
    
    
    COSINE_1 = float(merged.iloc[0][1])
    
    CALL_CENTER_ID_1 = float(merged.iloc[0][2])
     
    
    APP_DATE_1 = merged.iloc[0][17]
    
    ACTION_1 = merged.iloc[0][18]
     
   
     
   
   
    return (COSINE_1, CALL_CENTER_ID_1, APP_DATE_1, ACTION_1)

In [5]:
for i in range(2):
    print(execute(**df_scoring.iloc[i]))


(1.0, 576937.0, '2020-03-03', 'Verify Account Info')
(0.9999999999999999, 398647.0, '2020-03-03', 'Dismiss')
